## 対数線形モデル(クロス表の分析)

- 概要
    - 質的変数間の連関
    - 用いるデータの概要と分析目的
- 飽和モデルの分析
- 独立モデルの分析
- 最良モデルの探索
- 対数線形モデルの報告例
- 対数線形モデルとポアソン分布
- 逸脱度
- モデルの自由度
- 逸脱度を用いた尤度比検定
- 母数の制約
- 母数と期待度数
- 期待度数と関連付けた母数の解釈
    - 切片の解釈
    - 主効果の解釈
    - 1次の交互作用効果の解釈
    - 1次の交互作用効果の別の求め方
    - 2次の交互作用効果の解釈
    - より高次の交互作用効果
- 基準セルの設定    

### 概要

#### 質的変数間の連関

- 質的変数間の連関の検討はクロス集計表に対する$\chi^2$検定を用いる。  
- 対数線形モデルは、質的変数のカテゴリの組み合わせの効果の観点から変数間の連関を詳細に分析する。  
- 対数線形モデルは3変数以上の連関について一度の分析で検討でき、カテゴリの組み合わせ効果(交互作用効果)の有無について分析者の仮説を反映させたモデルを複数作成し、データへの適合を比較検討することが可能。

#### 用いるデータの概要と分析目的

##### イタリア老舗自転車メーカー3社の利用ユーザーに関する調査

##### データ概要

|変数名|説明|
 |:---|:---|
 |年代|20代、30代、40代|
 |性別|M=男性、F=女性|
 |メーカー|コレナゴ、デロンザ、ピロリロ|
|度数|回答者数|  

In [1]:
#データ読み込み・確認

#集計結果のデータ(個票データではない)
bdat <- read.csv("https://raw.githubusercontent.com/gndb3168/Renshu_data/master/R_MA/Sec_10/bicycle.csv") 
dim(bdat)
str(bdat)
head(bdat)

[1] 18  4

'data.frame':	18 obs. of  4 variables:
 $ 年代    : Factor w/ 3 levels "20代","30代",..: 1 2 3 1 2 3 1 2 3 1 ...
 $ 性別    : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 1 ...
 $ メーカー: Factor w/ 3 levels "コレナゴ","デロンザ",..: 1 1 1 2 2 2 3 3 3 1 ...
 $ 度数    : int  510 649 1598 554 645 1658 744 1623 2433 794 ...


年代,性別,メーカー,度数
20代,M,コレナゴ,510
30代,M,コレナゴ,649
40代,M,コレナゴ,1598
20代,M,デロンザ,554
30代,M,デロンザ,645
40代,M,デロンザ,1658


In [2]:
#集計データからのクロス集計表の作成

#行方向に年代、列方向にメーカーにしてクロス集計(行名・列名をつけるために用いるだけ)
tmpm <- table(bdat$年代, bdat$メーカー)

#男性におけるクロス集計表の作成
mm <- matrix(bdat$度数[1:9], ncol=3, nrow=3)
colnames(mm) <- colnames(tmpm)
rownames(mm) <- rownames(tmpm)

#女性におけるクロス集計表の作成
fm <- matrix(bdat$度数[10:18], ncol=3, nrow=3)
colnames(fm) <- colnames(tmpm)
rownames(fm) <- rownames(tmpm)

mm
fm

,コレナゴ,デロンザ,ピロリロ
20代,510,554,744
30代,649,645,1623
40代,1598,1658,2433


,コレナゴ,デロンザ,ピロリロ
20代,794,804,1243
30代,987,1055,2517
40代,1528,1519,2255


##### クロス集計表の問題点
2次以上の交互作用(3変数以上の連関)については言及しにくい。

#### 対数線形モデルの飽和モデルのモデル式(40代・男性・「ピロリロ」と回答した人の期待度数の対数)
$$
    \mbox{log}(m_{M40 \small{\mbox{ピロ}}} \ ) \  = \  \mu + \alpha_{40} + \beta_M + \delta_{\small{\mbox{ピロ}}} + (\alpha \delta)_{40M}
        + (\alpha \delta)_{40 \small{\mbox{ピロ}}} + (\beta \delta)_{M \small{\mbox{ピロ}}} 
        + + (\alpha \beta \delta)_{40M \small{\mbox{ピロ}}} 
$$

飽和モデルには主効果と1次の交互作用効果と2次の交互作用効果をモデル式に含んでいる。  
※$\alpha \beta$等は1次の交互作用効果の母数を示しており、2つの母数の積ではないので注意。  
4変数以上で定義される多重クロス集計表に対して対数線形モデルを用いた場合はオッズ比に基づく交互作用効果の解釈が非常に困難になる。  

#### 対数線形モデルの独立モデルのモデル式(40代・男性・「ピロリロ」と回答した人の期待度数の対数)
$$
    \mbox{log}(m_{M40 \small{\mbox{ピロ}}} \ ) \  = \  \mu + \alpha_{40} + \beta_M + \delta_{\small{\mbox{ピロ}}}
$$

独立モデルは主効果と切片のみの式。  
独立モデルが採択された場合、質的変数間に連関がないことが示唆される。  
独立モデルが棄却された場合は、飽和モデルから母数を減らし、度数の説明において特に重要な交互作用項のみを含んだ最良モデルを探索する。

#### 対数線形モデルの分析手順
1. 飽和モデルに基づく分析と解釈
1. 独立モデルに基づく分析と飽和モデルとの結果比較・解釈
1. 最良モデルの探索と解釈

### 10.2 飽和モデルの分析

In [3]:
#飽和モデルの分析

#度数の予測にはポアソン分布のモデル式を用いる
fullmodel <- glm(度数~年代*性別*メーカー, data=bdat, family="poisson")　#'*'を用いることで交互作用項もモデルに含めることが出来る
summary(fullmodel)


Call:
glm(formula = 度数 ~ 年代 * 性別 * メーカー, family = "poisson", 
    data = bdat)

Deviance Residuals: 
 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      6.67708    0.03549 188.147  < 2e-16 ***
年代30代                         0.21759    0.04767   4.564 5.01e-06 ***
年代40代                         0.65463    0.04375  14.964  < 2e-16 ***
性別M                           -0.44267    0.05675  -7.801 6.15e-15 ***
メーカーデロンザ                 0.01252    0.05003   0.250    0.802    
メーカーピロリロ                 0.44820    0.04543   9.866  < 2e-16 ***
年代30代:性別M                   0.02344    0.07599   0.308    0.758    
年代40代:性別M                   0.48747    0.06709   7.266 3.69e-13 ***
年代30代:メーカーデロンザ        0.05411    0.06682   0.810    0.418    
年代40代:メーカーデロンザ       -0.01842    0.06177  -0.298    0.766    
年代30代:メーカーピロリロ        0.48795    0.05894   8.278  < 2e-16 ***
年代40代:メーカーピロリロ 

全てのカテゴリが記載されてるわけではない。  
これは、「基準セル」によって一部のカテゴリの水準の効果が0に固定し、他の水準の効果を相対的に求めているため。

Residual devianceはデータに対するモデルの逸脱度を示し、飽和モデルでは限りなく0に近づく。  
それに対しNull devianceは、切片のみのモデルの逸脱度を示しており、相対的に適合が悪いことが示唆されている。

### 10.3 独立モデルの分析

In [4]:
#独立モデルの分析

idmodel <- glm(度数~年代+性別+メーカー, data=bdat, family="poisson")
summary(idmodel)


Call:
glm(formula = 度数 ~ 年代 + 性別 + メーカー, family = "poisson", 
    data = bdat)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-11.1299   -2.7982   -0.9488    3.7987   12.9502  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)       6.50782    0.01929 337.342   <2e-16 ***
年代30代          0.47505    0.01868  25.434   <2e-16 ***
年代40代          0.86042    0.01750  49.180   <2e-16 ***
性別M            -0.19861    0.01322 -15.024   <2e-16 ***
メーカーデロンザ  0.02748    0.01803   1.524    0.128    
メーカーピロリロ  0.57823    0.01604  36.047   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 5390.57  on 17  degrees of freedom
Residual deviance:  715.64  on 12  degrees of freedom
AIC: 887.37

Number of Fisher Scoring iterations: 4


In [5]:
#anovaを用いた飽和モデルと独立モデルの尤度比検定

anova(idmodel, fullmodel, test="Chisq")

Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
12,7.156365e+02,NA,NA,NA
0,-2.389200e-13,12,715.6365,1.980284e-145


In [6]:
#飽和モデルと独立モデルのAICとBIC
extractAIC(fullmodel)
extractAIC(idmodel)
extractAIC(fullmodel, k=log(sum(bdat$度数)))
extractAIC(idmodel, k=log(sum(bdat$度数)))

[1]  18.000 195.729

[1]   6.0000 887.3656

[1]  18.0000 340.5981

[1]   6.0000 935.6552

### 10.4 最良モデルの探索

#### 最良モデルの探索のルール
- 主効果は必ずモデルに含む(独立モデルが最も制約が強いモデル)
- 交互作用効果をモデルに含める場合は、下位の交互作用効果をすべて含める

In [7]:
#提案モデルの分析

#飽和モデルで有意な効果のみモデルに含める
bestmodel <- glm(度数~年代+性別+メーカー+(年代:性別)+(年代:メーカー), data=bdat, family="poisson")
summary(bestmodel)


Call:
glm(formula = 度数 ~ 年代 + 性別 + メーカー + (年代:性別) + (年代:メーカー), 
    family = "poisson", data = bdat)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.04060  -0.34486   0.01281   0.39917   1.11682  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                6.680696   0.030063 222.226  < 2e-16 ***
年代30代                   0.224719   0.040007   5.617 1.94e-08 ***
年代40代                   0.637821   0.036349  17.547  < 2e-16 ***
性別M                     -0.451935   0.030085 -15.022  < 2e-16 ***
メーカーデロンザ           0.040577   0.038772   1.047    0.295    
メーカーピロリロ           0.421190   0.035639  11.818  < 2e-16 ***
年代30代:性別M             0.005387   0.038305   0.141    0.888    
年代40代:性別M             0.522385   0.035630  14.662  < 2e-16 ***
年代30代:メーカーデロンザ -0.002203   0.051988  -0.042    0.966    
年代40代:メーカーデロンザ -0.024393   0.046238  -0.528    0.598    
年代30代:メーカーピロリロ  0.507252   0.046075  11.009  < 2e-16 ***
年代40代:メーカー

In [8]:
#飽和モデルと提案モデルの尤度比検定
anova(bestmodel, fullmodel, test="Chisq")

Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
6,5.811422e+00,NA,NA,NA
0,-2.389200e-13,6,5.811422,0.444643


In [9]:
#提案モデルと飽和モデルのAICとBIC
extractAIC(fullmodel)
extractAIC(bestmodel)
extractAIC(fullmodel, k=log(sum(bdat$度数)))
extractAIC(bestmodel, k=log(sum(bdat$度数)))

[1]  18.000 195.729

[1]  12.0000 189.5405

[1]  18.0000 340.5981

[1]  12.0000 286.1198

### 10.5 対数線形モデルの報告例

　「年代」「性別」「メーカー」の3変数間の連関構造を検討するために, 対数線形モデルによる分析を行った. 基準セルを"20代女性のコレナゴユーザー"としたうえで, 主効果と交互作用効果の推定を行った.  
　飽和モデルと独立モデルを算出し, 両モデルの適合度について逸脱度の差に基づく尤度比検定を行ったところ, 飽和モデルの相対的な適合の高さが示された($\chi^2(12) = 715.64, p < 0.001$). また, 飽和モデルにおいてはAIC = 195.729,  独立モデルにおいてはAIC=887.366, BIC=935.655であり, 情報量規準の観点からも飽和モデルの相対的な適合の高さが示された.  
 　上述の結果から, 独立モデルでは制約が強すぎる可能性が示唆されたので, 次に, 飽和モデルに含まれる主効果と交互作用の検定結果に基づいて, 「年代」「性別」「メーカー」の主効果, 「年代」「性別」の交互作用効果, 「年代」「メーカー」の交互作用効果を含むモデルを提案モデルとした.  
  尤度比検定を用いて飽和モデルと提案モデルとの適合度の比較を行ったところ, 有意差は見られなかった($\chi^2(6)=5.811, n.s.$). また, 提案モデルにおいてAIC=189.541, BIC=286.120であり, 情報量規準の観点からは提案モデルの相対的適合の高さが示された. この分析結果から, 提案モデルを最良モデルとして採択した. 下表に母数の推定結果を報告する.

\begin{array}{llc|llc} \hline
    母数 & 推定値 & 標準誤差 & 母数 & 推定値 & 標準偏差 \\ \hline
            \mu & 6.681*** & 0.030 & (\alpha \beta)_{30M} & 0.005 & 0.038 \\
            \alpha_{30} & 0.225*** & 0.040 & (\alpha \beta)_{40M} & 0.522*** & 0.036 \\
            \alpha_{40} & 0.638*** & 0.036 & (\alpha \delta)_{\text{30デロ}} & -0.002 & 0.052 \\
            \beta_M & -0.452*** & 0.030 & (\alpha \delta)_{\text{40デロ}} & -0.024 & 0.046 \\
            \delta_{デロ} & 0.041 & 0.039 & (\alpha \delta)_{\text{30ピロ}} & -0.507*** & 0.046 \\
            \delta_{ピロ} & 0.421*** & 0.036 & (\alpha \delta)_{\text{40ピロ}} & -0.016 & 0.042 \\ \hline              
\end{array}
\begin{array}{l}
    *:p < 0.05, \  **: p < 0.01, \  ***: p < 0.001
\end{array}

「年代」の主効果については, $\alpha_{30} = 0.025$, $\alpha_{40}$であり, それぞれ0.1%水準で有意であった. 20代を基準とすると, 30代, 40代と年齢が上昇するとともに, 主効果が増加する傾向がみられた.

「性別」の主効果については$\beta_M = -0.452$であり, 0.1%水準で有意であった. 女性に比較して男性の主効果が小さい傾向がうかがえた. また, 「メーカー」の主効果については$\delta_{\text{ピロ}}(=0.421)$のみが0.1%水準で有意であり, コレナゴに比較してピロリロの主効果が大きい傾向が示された.

「年代」と「性別の1次の交互作用効果と, 「年代」と「メーカー」の1次の交互作用効果もそれぞれ有意であった. 「年代」と「性別」については, 40代の男性の交互作用効果が$(\alpha \beta)_{40M} = 0.552$であり, 0.1%水準で有意であった. 40代で男性という属性のセルの度数は, この正の交互作用効果によってある程度説明されることが明らかになった. また, 「年代」と「メーカー」については, 30代のピロリロユーザーの交互作用効果が$(\alpha \beta)_{30 \text{ピロ}} = 0.507$であり, 0.1%水準で有意であった.l 性別を問わず, 30代のユーザーにはピロリロの人気が高いことが示唆される結果となった.

### 10.6 対数線形モデルとポアソン分布

対数線形モデルはクロス集計表の度数($n_{ij}$)をポアソン分布に従うと仮定する.  
確率関数とパラメータ($m_{ij}$)は以下のようになる.

$$
    f(n_{ij}) = \frac{m_{ij}^{n_{ij}}\textrm{exp}(-m_{ij})}{n_{ij}!} \\
    E[n_{ij}] = m_{ij}
$$

二重クロス集計表における対数線形モデルの飽和モデル

$$
    \textrm{log}\, E[n_{ij}] = \textrm{log}(m_{ij}) = \mu + \alpha_i + \beta_j + (\alpha + \beta)_{ij}
$$

データの総数をあらかじめ定めずサンプリングした場合は各セルの度数は互いに独立.  
全セルの度数を含んだベクトル$\boldsymbol{n}(= n_{11}, \ldots, n_{IJ})'$の同時分布(尤度関数)は

$$
    f(\boldsymbol{n}) = \prod_{i=1}^I \prod_{j=1}^J \frac{m_{ij}^{n_{ij}}\textrm{exp}(-m_{ij})}{n_{ij}!}
$$

積の演算は計算上の問題が生じやすいので, 対数尤度関数を微分して母数推定する.  

$$
    \textrm{log}\, L(\boldsymbol{n}) = \sum_{i=1}^I \sum_{j=1}^J \{n_{ij}\, \textrm{log}\, m_{ij} - m{ij} - \textrm{log}(n_{ij}!) \}
$$

$L(\boldsymbol{n})$は$f(\boldsymbol{n})$に対応する尤度

### 10.7 逸脱度

- 逸脱度(deviance)は, データに対するモデルの適合度
- 飽和モデルはデータに対して完全に適合
- 飽和モデルの対数尤度と提案モデルの対数尤度の差が逸脱度の値

##### 2重クロス集計表に対する対数線形モデルにおける逸脱度
$$
    \text{(提案モデルの逸脱度)} = 2 \sum_{i=1}^I \sum_{j=1}^J n_{ij} \log \left(\frac{n_{ij}}{m_{ij}}\right)   
$$

提案モデルが飽和モデルであった場合, 観測度数と期待度数は一致し, $m_{ij} = n_{ij}$となり, 逸脱度は$\log(1)=0$になる.

### 10.8 モデルの自由度

$$
    \text(自由度) = \text{セルの個数} - \text{推定する母数の個数}   
$$

### 10.9 逸脱度を用いた尤度比検定

- 標本サイズが十分に大きい場合, 対数線形モデルにおけるモデルAとモデルBとの逸脱度の差は自由度が(自由度A $-$ 自由度B)の$\chi^2$分布に近似
- 逸脱度の差の尤度比検定は逸脱度の差の$\chi^2$検定

### 10.10 母数の制約

- 対数線形モデルにおいて, 「基準セル」を設定することで関連する主効果と交互作用効果を$0$に制約する母数の端点制約を行う
- 端点制約を置かない場合, 1つの母数に対して無数に推定値の候補が生じ, 解が一位に定まらない
- 端点制約を置くことで推定する母数が減り, 期待度数のオッズと関連付けて母数を解釈しやすい

サンプルデータでは「20代女性のコレナゴユーザー」が基準セルになっている.

### 10.11 母数と期待度数

- サンプルデータにおける基準セルは「20代・女性・コレナゴユーザー」
- 期待度数$\log \left(m_{20F\text{コレ}}\  \right)$のモデルは, 主効果と交互作用効果が全て0であるため, 切片$\mu$のみ

女性における「年代」と「メーカー」のモデル構造

$
\begin{align}
    \log \left(m_{20F\text{コレ}}\  \right) & = \mu \\
    \log \left(m_{30F\text{コレ}}\  \right) & = \mu + \alpha_{30} \\
    \log \left(m_{40F\text{コレ}}\  \right) & = \mu + \alpha_{40} \\
    \log \left(m_{20F\text{デロ}}\  \right) & = \mu + \delta_{\text{デロ}} \\
    \log \left(m_{30F\text{デロ}}\  \right) & = \mu + \alpha_{30} + \delta_{\text{デロ}} + (\alpha \delta)_{\text{30デロ}} \\
    \log \left(m_{40F\text{デロ}}\  \right) & = \mu + \alpha_{40} + \delta_{\text{デロ}} + (\alpha \delta)_{\text{40デロ}} \\
    \log \left(m_{20F\text{ピロ}}\  \right) & = \mu + \delta_{\text{ピロ}} \\
    \log \left(m_{30F\text{ピロ}}\  \right) & = \mu + \alpha_{30} + \delta_{\text{ピロ}} + (\alpha \delta)_{\text{30ピロ}} \\
    \log \left(m_{40F\text{ピロ}}\  \right) & = \mu + \alpha_{40} + \delta_{\text{ピロ}} + (\alpha \delta)_{\text{40ピロ}} 
\end{align}
$

In [10]:
#最良モデルの期待度数行列

#fitted.valuesはモデルで算出した推定値
xtabs(bestmodel$fitted.values~bdat$年代+bdat$メーカー+bdat$性別)

, , bdat$性別 = F

         bdat$メーカー
bdat$年代  コレナゴ  デロンザ  ピロリロ
     20代  796.8733  829.8727 1214.2540
     30代  997.6624 1036.6907 2524.6469
     40代 1507.9658 1532.5679 2261.4663

, , bdat$性別 = M

         bdat$メーカー
bdat$年代  コレナゴ  デロンザ  ピロリロ
     20代  507.1267  528.1273  772.7460
     30代  638.3376  663.3093 1615.3531
     40代 1618.0342 1644.4321 2426.5337
